In [2]:
import tensorflow_decision_forests as tfdf
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

Dataset

In [ ]:
# Load a dataset into a Pandas Dataframe.
df = pd.read_csv("FertilizerPrediction.csv")

# Rename the column 'Fertilizer Name' to 'New Column Name'
df = df.rename(columns={'Temparature': 'temparature', 'Humidity ': 'humidity', 'Moisture': 'moisture', 'Soil Type': 'soil_type' , 'Crop Type': 'crop_type', 'Nitrogen': 'nitrogen', 'Potassium' : 'potassium', 'Phosphorous': 'phosphorous','Fertilizer Name': 'fertilizer'})

# Display the first 3 examples.
df.head(3)

Encode

In [ ]:
# Encode the categorical labels as integers.
#
# Details:
# This stage is necessary if your classification label is represented as a
# string since Keras expects integer classification labels.
# When using `pd_dataframe_to_tf_dataset` (see below), this step can be skipped.

# Name of the label column.
label = "fertilizer"

classes = df[label].unique().tolist()
print(f"Label classes: {classes}")

df[label] = df[label].map(classes.index)
df.head(3)

Split dataset

In [ ]:
# Split the dataset into a training and a testing dataset.

def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

In [ ]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label)

In [ ]:
# Specify the model.
model_1 = tfdf.keras.RandomForestModel(verbose=2)

# Train the model.
model_1.fit(train_ds)

In [ ]:
model_1.compile(metrics=["accuracy"])
evaluation = model_1.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

In [ ]:
model_1.make_inspector().training_logs()

Visualize Model

In [ ]:
import matplotlib.pyplot as plt

logs = model_1.make_inspector().training_logs()

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Accuracy (out-of-bag)")

plt.subplot(1, 2, 2)
plt.plot([log.num_trees for log in logs], [log.evaluation.loss for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Logloss (out-of-bag)")

plt.show()

In [ ]:
tfdf.keras.get_all_models()

In [ ]:
import dtreeviz

from matplotlib import pyplot as plt
from IPython import display

# avoid "Arial font not found warnings"
import logging
logging.getLogger('matplotlib.font_manager').setLevel(level=logging.CRITICAL)

display.set_matplotlib_formats('retina') # generate hires plots

np.random.seed(1234)  # reproducible plots/data for explanatory reasons

In [ ]:
# Tell dtreeviz about training data and model
fertilizer_features = [f.name for f in model_1.make_inspector().features()]
viz_cmodel = dtreeviz.model(model_1,
                           tree_index=3,
                           X_train=train_ds_pd[fertilizer_features],
                           y_train=train_ds_pd[label],
                           feature_names=fertilizer_features,
                           target_name=label,
                           class_names=classes)

In [ ]:
viz_cmodel.view(scale=1.2)

In [ ]:
viz_cmodel.view(leaftype='barh', scale=.75)

In [ ]:
x = train_ds_pd[fertilizer_features].iloc[20]
viz_cmodel.view(x=x, scale=.75)

Save model

In [ ]:
# VERSION = 1
# !mkdir -p save_model/{VERSION}
# tf.keras.models.save_model(model_1, os.path.join('save_model', str(VERSION)))

VERSION = 1
save_path = '/save_model'  # Specify the absolute path to the "models/" folder

# Create the directory if it doesn't exist
os.makedirs(os.path.join(save_path, str(VERSION)), exist_ok=True)

# Save the model to the specified path
tf.keras.models.save_model(model_1, os.path.join(save_path, str(VERSION)))


Predict

# Load the saved model
model = tf.keras.models.load_model('save_model/1')

# Prepare the input data for prediction
input_data = {
    'crop_type': tf.constant(['Maize']),
    'humidity': tf.constant([26], dtype=tf.int64),
    'moisture': tf.constant([52], dtype=tf.int64),
    'nitrogen': tf.constant([38], dtype=tf.int64),
    'phosphorous': tf.constant([0], dtype=tf.int64),
    'potassium': tf.constant([0], dtype=tf.int64),
    'soil_type': tf.constant(['Black']),
    'temparature': tf.constant([37], dtype=tf.int64),
}

predictions = model.predict(input_data)
predicted_label = tf.argmax(predictions, axis=1)[0].numpy()

# Define the label mappings
label_mappings = {
    0: "10-26-26",
    1: "14-35-14",
    2: "17-17-17",
    3: "20-20",
    4: "28-28",
    5: "DAP",
    6: "Urea"
}

# Print the predicted label
if predicted_label in label_mappings:
    print(label_mappings[predicted_label])
else:
    print("Unknown label")